In [9]:
# !pip install nest_asyncio
# !pip install newspaper3k feedparser psycopg2
import nest_asyncio
nest_asyncio.apply()

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
import pandas as pd
df = pd.read_csv('/workspace/twint/rss/resource/rss.csv')
for index, r in df.iterrows():
    print(r['ticker'], r['url'])

yhoo http://finance.yahoo.com/rss/headline?s=yhoo
tivo http://finance.yahoo.com/rss/headline?s=tivo


In [16]:
import aiohttp
import feedparser

headers = {
#     "If-None-Match": "246ce00049b1df9930d4dc344180574f"
}

async def get():
    async with aiohttp.ClientSession() as session:
#         async with session.get('http://finance.yahoo.com/rss/industry?s=yhoo', headers=headers) as resp:
        async with session.get('http://finance.yahoo.com/rss/headline?s=dis', headers=headers) as resp:
            print(resp.status)
            print(resp.headers)
            t = await resp.text()
            return feedparser.parse(t)

await get()

200
<CIMultiDictProxy('X-Content-Type-Options': 'nosniff', 'X-Yahoo-Request-Id': 'fuuk2p1f8g7n2', 'Cache-Control': 'max-age=300, stale-while-revalidate=75', 'Content-Encoding': 'gzip', 'Content-Type': 'text/xml;charset=utf-8', 'Vary': 'Origin', 'Content-Length': '5707', 'Date': 'Sat, 04 Apr 2020 05:45:06 GMT', 'Age': '0', 'Strict-Transport-Security': 'max-age=31536000', 'Server': 'ATS', 'Expect-CT': 'max-age=31536000, report-uri="http://csp.yahoo.com/beacon/csp?src=yahoocom-expect-ct-report-only"', 'Public-Key-Pins-Report-Only': 'max-age=2592000; pin-sha256="2fRAUXyxl4A1/XHrKNBmc8bTkzA7y4FB/GLJuNAzCqY="; pin-sha256="I/Lt/z7ekCWanjD0Cvj5EqXls2lOaThEA0H2Bg4BT/o="; pin-sha256="K87oWBWM9UZfyddvDfoxL+8lpNyoUB2ptGtn0fv6G2Q="; pin-sha256="Wd8xe/qfTwq3ylFNd3IpaqLHZbh2ZNCLluVzmeNkcpw="; pin-sha256="WoiWRyIOVNa9ihaBciRSC7XHjliYS9VwUGOIud4PB18="; pin-sha256="cGuxAXyFXFkWm61cF4HPWX8S0srS9j0aSqN0k4AP+4A="; pin-sha256="dolnbtzEBnELx/9lOEQ22e6OZO/QNb6VSSX2XHA3E7A="; pin-sha256="i7WTqTvh0OioIruIfFR4kM

{'feed': {'rights': 'Copyright (c) 2020 Yahoo! Inc. All rights reserved.',
  'rights_detail': {'type': 'text/plain',
   'language': None,
   'base': '',
   'value': 'Copyright (c) 2020 Yahoo! Inc. All rights reserved.'},
  'subtitle': 'Latest Financial News for dis',
  'subtitle_detail': {'type': 'text/html',
   'language': None,
   'base': '',
   'value': 'Latest Financial News for dis'},
  'image': {'height': 45,
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'http://finance.yahoo.com/q/h?s=dis'}],
   'link': 'http://finance.yahoo.com/q/h?s=dis',
   'title': 'Yahoo! Finance: dis News',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': '',
    'value': 'Yahoo! Finance: dis News'},
   'href': 'http://l.yimg.com/a/i/brand/purplelogo/uh/us/fin.gif',
   'width': 144},
  'language': 'en-US',
  'updated': 'Sat, 04 Apr 2020 05:45:06 +0000',
  'updated_parsed': time.struct_time(tm_year=2020, tm_mon=4, tm_mday=4, tm_hour=5, tm_min=45, tm_sec=

In [11]:
import time
import datetime

t = time.localtime()
datetime.datetime.fromtimestamp(time.mktime(t))

datetime.datetime(2020, 4, 15, 10, 12, 53)

In [7]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

async def say():
    await asyncio.sleep(-3)
    print("hello")
    
async def main():
    await asyncio.gather(
        book(),
        book()
    )
#     loop = asyncio.get_running_loop()
#     task = loop.create_task(say())

asyncio.run(main())
# loop = asyncio.get_event_loop()

hello
hello


In [12]:
f = feedparser.parse('http://finance.yahoo.com/rss/headline?s=dis')

In [26]:
import json
import pprint

a = '''
[{"summary": "Futures fell early Wednesday as Tesla and airlines rallied. The coronavirus stock rally soared Tuesday as Amazon, Microsoft and Nvidia led breakouts. Apple and the Nasdaq cleared key levels.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "Futures fell early Wednesday as Tesla and airlines rallied. The coronavirus stock rally soared Tuesday as Amazon, Microsoft and Nvidia led breakouts. Apple and the Nasdaq cleared key levels."}, "id": "8288ce8e-17d6-3d23-9871-7adf026869b5", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/8288ce8e-17d6-3d23-9871-7adf026869b5/dow-jones-futures-fall-but.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/8288ce8e-17d6-3d23-9871-7adf026869b5/dow-jones-futures-fall-but.html?.tsrc=rss", "published": "Wed, 15 Apr 2020 11:39:02 +0000", "published_parsed": [2020, 4, 15, 11, 39, 2, 2, 106, 0], "title": "Dow Jones Futures Fall But Tesla, Airlines Rise After Amazon Leads New Breakouts In Coronavirus Stock Market Rally", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "Dow Jones Futures Fall But Tesla, Airlines Rise After Amazon Leads New Breakouts In Coronavirus Stock Market Rally"}}, {"summary": "(Bloomberg) -- The lockdown gripping much of the world economy has spurred a real-time stress test of the long-heralded digital future.Virtual brown bag lunches have replaced office gatherings; schools have rushed out internet-based learning; the International Monetary Fund will this week hold its Spring meetings online; and the U.S. Supreme Court for the first time will hear arguments by telephone and allow live audio broadcasts.Virus lockdowns have seen millions lose their jobs as waiters, flight attendants, Pilates instructors and other service providers are shuttered. That means sustaining those sectors that can function online has never been more important for a global economy facing one of its darkest periods since the Great Depression.Guaging the exact size of the digital economy isn\u2019t easy: The United Nations Conference on Trade and Development estimates a range of 4.5% to 15.5% of global gross domestic product. While that varies by country, it\u2019s clear the companies and nations which can migrate the most commerce online will go some way to cushioning the damage.\u201cThe futurists are going to have a field day,\u201d said Mark Herlach an international lawyer at Eversheds Sutherland LLP in Washington D.C. \u201cIt will change the way we build our cities, the way we move around in those cities and that in turn changes our energy use. A whole series of knock on effects are coming.\u201dHerlach -- who has had to steer negotiations between clients and government from his home and has used video happy hours and dinner parties to stay in touch with colleagues and friends -- is positive on the experience so far, but worries about a lingering sense of isolation if more opt to work remotely once the lockdowns ease.Cheaper internet connectivity has enabled explosive growth in online tools, allowing many white collar roles to be done at home and keeping managers and business owners in touch with their staff.Users of Microsoft Teams soared to a new daily record of 2.7 billion meeting minutes in one day, a 200% increase from 900 million on March 16, the company said on April 9. Even amid security concerns, Zoom has gone from being used by 10 million office workers a day to more than 200 million people.\u201c\u2018Zooming\u2019 has become a new verb,\u201d said Michael Bowes, a barrister and joint head of Outer Temple Chambers in London. He and his colleagues hold a \u2018Virtual Tea Zoom Group\u2019 at 4pm every Wednesday, where everyone brings their own tea and cake for a general chat about non-work issues.Already, some companies have an eye on how they\u2019ll change operations even when the virus dissipates. Some are looking at cutting expensive travel and real estate budgets in favor of investing in better technology and home office set ups, said Satish Shankar, regional managing partner of Bain & Company Asia-Pacific.\u201cWe are poised for a dramatic wanton increase of the digitalization of our economies,\u201d he said.Viewed through metrics such as online government services, fiber internet connection and the share of people who already work from home, Scandinavian nations score highly in terms of online readiness. In tech rich economies such as Japan and South Korea, the sectors most impacted can\u2019t readily switch to a remote stance, according to London based HSBC economist James Pomeroy.Ed Yardeni, who coined the term \u201cbond vigilantes\u201d in the 1980s, dedicated a recent research note to clients on how he has adjusted to a world of meetings and cocktails over video link during what he calls the great virus crisis, or GVC.\u201cTechnology has become a GVC staple, right up there with food and toilet paper,\u201d wrote the president and chief investment strategist of Yardeni Research Inc.Perhaps no sector has been as upended as education, with school closures affecting 90% of the world\u2019s students, or more than 1.5 billion people according to UNESCO. That has forced teachers to scramble online courses with little warning, with knock on consequences for the wider economy as parents are forced to adapt their working schedule around their children\u2019s needs.Exam timetables have been threatened as authorities grapple with canceling or postponing critical final year exams, or basing grades on past performance.Wong Mo Yee, a primary school teacher and a member of the Hong Kong Professional Teachers\u2019 Union executive committee where schools have been shut for months, said the crisis highlighted the need for clear goals about what should be taught at home instead of rigidly sticking with the in-class room curriculum. She also flagged dangers of too much screen time.\u201cHome learning is completely different, the interaction is different, the dynamic in the so-called video classes is also different,\u201d Wong said. \u201cIt\u2019s not so easy to engage students in video teaching.\u201dPoorer children have been hardest hit, where families either have to share devices or don\u2019t have them at all. Families where parents have to leave home for work and cannot supervise their child\u2019s learning have had it tougher still.\u201cThe longer children are out of school and not learning, the increased likelihood they will never return to school,\u201d said Heather Simpson, chief program officer of Room to Read.Tech LimitsManufacturers have also discovered the limitations of tech. One of them is Colin Ng, who is co-founder of Hong Kong-based Lincogn Technology Co. that designs and makes smart home appliances such as facial recognition door locks and mobile phone controlled lights for the Hong Kong, China, U.S. and European markets.Travel restrictions on the border between Hong Kong and mainland China have disrupted the main artery between Ng\u2019s small R&D team in Hong Kong and the company\u2019s manufacturing staff in neighboring Guangdong. That\u2019s complicated the process for when a product is meant to move from the R&D laboratory to the factory floor.\u201cVideo conferencing the discussion is very difficult,\u201d Ng said. \u201cIt is difficult to explain a lot of the detail through the camera, the conversation becomes very inefficient.\u201dJoerg Wuttke, president of the European Chamber of Commerce in China and a veteran of doing business in the world\u2019s second-biggest economy, said remote working can only do so much for those in due diligence, sales or manufacturing.\u201cCompanies are pushing the envelope,\u201d Wuttke said. \u201cBut at the end of the day someone still has to get the coal and oil out of the ground and put solar panels together.\u201dThere have been other strains too.Surging web usage prompted President Donald Trump to hold talks with telecom giants to ensure the networks could cope. In Europe, Amazon.com Inc., Netflix Inc. and YouTube had to reduce the quality of their video streams to ensure networks can handle increased usage.In China, mobile broadband downloading speed slumped between mid-January and early February before gradually recovering by mid-March, according to network speed testing platform Ookla. Downloading speed in India and Malaysia has started to dip since early March as the pandemic outbreak spread.The shift online has been a crucial safety net for a collapsing world economy and will change how we study, work and play even when the virus passes, said Chua Hak Bin, senior economist at Maybank Kim Eng Research Pte. in Singapore.\u201cQuarantines and lockdowns for such long periods would have been near impossible to impose and bear without the tech devices available today,\u201d he said.For more articles like this, please visit us at bloomberg.comSubscribe now to stay ahead with the most trusted business news source.\u00a92020 Bloomberg L.P.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "(Bloomberg) -- The lockdown gripping much of the world economy has spurred a real-time stress test of the long-heralded digital future.Virtual brown bag lunches have replaced office gatherings; schools have rushed out internet-based learning; the International Monetary Fund will this week hold its Spring meetings online; and the U.S. Supreme Court for the first time will hear arguments by telephone and allow live audio broadcasts.Virus lockdowns have seen millions lose their jobs as waiters, flight attendants, Pilates instructors and other service providers are shuttered. That means sustaining those sectors that can function online has never been more important for a global economy facing one of its darkest periods since the Great Depression.Guaging the exact size of the digital economy isn\u2019t easy: The United Nations Conference on Trade and Development estimates a range of 4.5% to 15.5% of global gross domestic product. While that varies by country, it\u2019s clear the companies and nations which can migrate the most commerce online will go some way to cushioning the damage.\u201cThe futurists are going to have a field day,\u201d said Mark Herlach an international lawyer at Eversheds Sutherland LLP in Washington D.C. \u201cIt will change the way we build our cities, the way we move around in those cities and that in turn changes our energy use. A whole series of knock on effects are coming.\u201dHerlach -- who has had to steer negotiations between clients and government from his home and has used video happy hours and dinner parties to stay in touch with colleagues and friends -- is positive on the experience so far, but worries about a lingering sense of isolation if more opt to work remotely once the lockdowns ease.Cheaper internet connectivity has enabled explosive growth in online tools, allowing many white collar roles to be done at home and keeping managers and business owners in touch with their staff.Users of Microsoft Teams soared to a new daily record of 2.7 billion meeting minutes in one day, a 200% increase from 900 million on March 16, the company said on April 9. Even amid security concerns, Zoom has gone from being used by 10 million office workers a day to more than 200 million people.\u201c\u2018Zooming\u2019 has become a new verb,\u201d said Michael Bowes, a barrister and joint head of Outer Temple Chambers in London. He and his colleagues hold a \u2018Virtual Tea Zoom Group\u2019 at 4pm every Wednesday, where everyone brings their own tea and cake for a general chat about non-work issues.Already, some companies have an eye on how they\u2019ll change operations even when the virus dissipates. Some are looking at cutting expensive travel and real estate budgets in favor of investing in better technology and home office set ups, said Satish Shankar, regional managing partner of Bain & Company Asia-Pacific.\u201cWe are poised for a dramatic wanton increase of the digitalization of our economies,\u201d he said.Viewed through metrics such as online government services, fiber internet connection and the share of people who already work from home, Scandinavian nations score highly in terms of online readiness. In tech rich economies such as Japan and South Korea, the sectors most impacted can\u2019t readily switch to a remote stance, according to London based HSBC economist James Pomeroy.Ed Yardeni, who coined the term \u201cbond vigilantes\u201d in the 1980s, dedicated a recent research note to clients on how he has adjusted to a world of meetings and cocktails over video link during what he calls the great virus crisis, or GVC.\u201cTechnology has become a GVC staple, right up there with food and toilet paper,\u201d wrote the president and chief investment strategist of Yardeni Research Inc.Perhaps no sector has been as upended as education, with school closures affecting 90% of the world\u2019s students, or more than 1.5 billion people according to UNESCO. That has forced teachers to scramble online courses with little warning, with knock on consequences for the wider economy as parents are forced to adapt their working schedule around their children\u2019s needs.Exam timetables have been threatened as authorities grapple with canceling or postponing critical final year exams, or basing grades on past performance.Wong Mo Yee, a primary school teacher and a member of the Hong Kong Professional Teachers\u2019 Union executive committee where schools have been shut for months, said the crisis highlighted the need for clear goals about what should be taught at home instead of rigidly sticking with the in-class room curriculum. She also flagged dangers of too much screen time.\u201cHome learning is completely different, the interaction is different, the dynamic in the so-called video classes is also different,\u201d Wong said. \u201cIt\u2019s not so easy to engage students in video teaching.\u201dPoorer children have been hardest hit, where families either have to share devices or don\u2019t have them at all. Families where parents have to leave home for work and cannot supervise their child\u2019s learning have had it tougher still.\u201cThe longer children are out of school and not learning, the increased likelihood they will never return to school,\u201d said Heather Simpson, chief program officer of Room to Read.Tech LimitsManufacturers have also discovered the limitations of tech. One of them is Colin Ng, who is co-founder of Hong Kong-based Lincogn Technology Co. that designs and makes smart home appliances such as facial recognition door locks and mobile phone controlled lights for the Hong Kong, China, U.S. and European markets.Travel restrictions on the border between Hong Kong and mainland China have disrupted the main artery between Ng\u2019s small R&D team in Hong Kong and the company\u2019s manufacturing staff in neighboring Guangdong. That\u2019s complicated the process for when a product is meant to move from the R&D laboratory to the factory floor.\u201cVideo conferencing the discussion is very difficult,\u201d Ng said. \u201cIt is difficult to explain a lot of the detail through the camera, the conversation becomes very inefficient.\u201dJoerg Wuttke, president of the European Chamber of Commerce in China and a veteran of doing business in the world\u2019s second-biggest economy, said remote working can only do so much for those in due diligence, sales or manufacturing.\u201cCompanies are pushing the envelope,\u201d Wuttke said. \u201cBut at the end of the day someone still has to get the coal and oil out of the ground and put solar panels together.\u201dThere have been other strains too.Surging web usage prompted President Donald Trump to hold talks with telecom giants to ensure the networks could cope. In Europe, Amazon.com Inc., Netflix Inc. and YouTube had to reduce the quality of their video streams to ensure networks can handle increased usage.In China, mobile broadband downloading speed slumped between mid-January and early February before gradually recovering by mid-March, according to network speed testing platform Ookla. Downloading speed in India and Malaysia has started to dip since early March as the pandemic outbreak spread.The shift online has been a crucial safety net for a collapsing world economy and will change how we study, work and play even when the virus passes, said Chua Hak Bin, senior economist at Maybank Kim Eng Research Pte. in Singapore.\u201cQuarantines and lockdowns for such long periods would have been near impossible to impose and bear without the tech devices available today,\u201d he said.For more articles like this, please visit us at bloomberg.comSubscribe now to stay ahead with the most trusted business news source.\u00a92020 Bloomberg L.P."}, "id": "beee8b30-c14b-3208-956a-3dda51ed1497", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/news/world-economy-working-home-gets-210000901.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/news/world-economy-working-home-gets-210000901.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 21:00:00 +0000", "published_parsed": [2020, 4, 14, 21, 0, 0, 1, 105, 0], "title": "World Economy Working From Home Gets a Glimpse of the Virtual Future", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "World Economy Working From Home Gets a Glimpse of the Virtual Future"}}, {"summary": "Investors have largely turned bullish on megacap tech names, even in the face of terrible economic and jobless data and a looming recession.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "Investors have largely turned bullish on megacap tech names, even in the face of terrible economic and jobless data and a looming recession."}, "id": "0dfc6f7a-e5e5-30db-8422-60eb05ddaf4f", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/0dfc6f7a-e5e5-30db-8422-60eb05ddaf4f/amazon-stock-hits-all-time.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/0dfc6f7a-e5e5-30db-8422-60eb05ddaf4f/amazon-stock-hits-all-time.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 20:52:00 +0000", "published_parsed": [2020, 4, 14, 20, 52, 0, 1, 105, 0], "title": "Amazon Stock Hits All-Time High, Up 24% This Year, as Shopping Moves Online", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "Amazon Stock Hits All-Time High, Up 24% This Year, as Shopping Moves Online"}}, {"summary": "The Dow Jones rose on the stock market today, while Amazon stock and Netflix stock lifted above their pre-coronavirus highs.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "The Dow Jones rose on the stock market today, while Amazon stock and Netflix stock lifted above their pre-coronavirus highs."}, "id": "e6d56c42-a977-3613-872c-bf7898f00349", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/e6d56c42-a977-3613-872c-bf7898f00349/dow-stocks-apple%2C-microsoft.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/e6d56c42-a977-3613-872c-bf7898f00349/dow-stocks-apple%2C-microsoft.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 20:30:58 +0000", "published_parsed": [2020, 4, 14, 20, 30, 58, 1, 105, 0], "title": "Dow Stocks Apple, Microsoft Surge As Nasdaq Leads Market", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "Dow Stocks Apple, Microsoft Surge As Nasdaq Leads Market"}}, {"summary": "On average, the most frequently owned shares largely fell less than the market on the way down this year, and matched its rise on the way back up.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "On average, the most frequently owned shares largely fell less than the market on the way down this year, and matched its rise on the way back up."}, "id": "c0ef6c1c-dcba-3660-b147-09442ed3af97", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/c0ef6c1c-dcba-3660-b147-09442ed3af97/hedge-funds%E2%80%99-top-holdings-are.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/c0ef6c1c-dcba-3660-b147-09442ed3af97/hedge-funds%E2%80%99-top-holdings-are.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 19:19:00 +0000", "published_parsed": [2020, 4, 14, 19, 19, 0, 1, 105, 0], "title": "Hedge Funds\u2019 Top Holdings Are Beating the Market. They\u2019re a Familiar Bunch.", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "Hedge Funds\u2019 Top Holdings Are Beating the Market. They\u2019re a Familiar Bunch."}}, {"summary": "DOW UPDATE Behind positive growth for shares of Apple Inc. and Johnson & Johnson, the Dow Jones Industrial Average is soaring Tuesday afternoon. Shares of Apple Inc. (AAPL) and Johnson & Johnson (JNJ) are contributing to the index&apos;s intraday rally, as the Dow (DJIA) is trading 494 points, or 2.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "DOW UPDATE Behind positive growth for shares of Apple Inc. and Johnson & Johnson, the Dow Jones Industrial Average is soaring Tuesday afternoon. Shares of Apple Inc. (AAPL) and Johnson & Johnson (JNJ) are contributing to the index&apos;s intraday rally, as the Dow (DJIA) is trading 494 points, or 2."}, "id": "ec147057-854b-36c6-a49f-216552d5f363", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/ec147057-854b-36c6-a49f-216552d5f363/apple-inc.%2C-johnson-%26-johnson.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/ec147057-854b-36c6-a49f-216552d5f363/apple-inc.%2C-johnson-%26-johnson.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 19:02:00 +0000", "published_parsed": [2020, 4, 14, 19, 2, 0, 1, 105, 0], "title": "Apple Inc., Johnson & Johnson share gains contribute to Dow&#39;s nearly 500-point surge", "title_detail": {"type": "text/html", "language": null, "base": "", "value": "Apple Inc., Johnson & Johnson share gains contribute to Dow&#39;s nearly 500-point surge"}}, {"summary": "DOW UPDATE Shares of Home Depot and Apple Inc. are posting positive growth Tuesday afternoon, sending the Dow Jones Industrial Average soaring. Shares of Home Depot (HD) and Apple Inc. (AAPL) have contributed around two thirds of the index&apos;s intraday rally, as the Dow (DJIA) is trading 556 points (2.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "DOW UPDATE Shares of Home Depot and Apple Inc. are posting positive growth Tuesday afternoon, sending the Dow Jones Industrial Average soaring. Shares of Home Depot (HD) and Apple Inc. (AAPL) have contributed around two thirds of the index&apos;s intraday rally, as the Dow (DJIA) is trading 556 points (2."}, "id": "5148d162-4e05-326a-857e-7c5565c147c5", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/5148d162-4e05-326a-857e-7c5565c147c5/dow%26%2339%3Bs-550-point-surge.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/5148d162-4e05-326a-857e-7c5565c147c5/dow%26%2339%3Bs-550-point-surge.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 16:57:00 +0000", "published_parsed": [2020, 4, 14, 16, 57, 0, 1, 105, 0], "title": "Dow&#39;s 550-point surge highlighted by gains for shares of Home Depot, Apple Inc.", "title_detail": {"type": "text/html", "language": null, "base": "", "value": "Dow&#39;s 550-point surge highlighted by gains for shares of Home Depot, Apple Inc."}}, {"summary": "Beating the first quarter's market crash took some extreme strategies, such as moving nearly everything to cash, or going very short.  Now, fund managers who prevailed during the worst start to a year for U.S. stocks are betting on recession-resilient companies to take them to the next leg.  With few expecting a quick end to the market's volatility, managers are seeking companies with strong balance sheets that can weather what could be a deep but short recession, according to a recent Reuters poll.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "Beating the first quarter's market crash took some extreme strategies, such as moving nearly everything to cash, or going very short.  Now, fund managers who prevailed during the worst start to a year for U.S. stocks are betting on recession-resilient companies to take them to the next leg.  With few expecting a quick end to the market's volatility, managers are seeking companies with strong balance sheets that can weather what could be a deep but short recession, according to a recent Reuters poll."}, "id": "0c9341d4-ea2d-3b69-983a-087ab106cf53", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/news/u-funds-beat-market-plunge-163353683.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/news/u-funds-beat-market-plunge-163353683.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 16:33:53 +0000", "published_parsed": [2020, 4, 14, 16, 33, 53, 1, 105, 0], "title": "U.S. funds that beat market plunge seek recession-proof stocks", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "U.S. funds that beat market plunge seek recession-proof stocks"}}, {"summary": "Morgan Stanley sees strong core secular growth in trends in software, with a particular focus on recurring revenue models, and sees benefits for the sector from the aggressive fiscal and monetary steps to fight the crisis.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "Morgan Stanley sees strong core secular growth in trends in software, with a particular focus on recurring revenue models, and sees benefits for the sector from the aggressive fiscal and monetary steps to fight the crisis."}, "id": "6d984fb5-126d-382d-ab2d-09912a6b8817", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/6d984fb5-126d-382d-ab2d-09912a6b8817/7-cloud-software-stocks-to.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/6d984fb5-126d-382d-ab2d-09912a6b8817/7-cloud-software-stocks-to.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 16:07:00 +0000", "published_parsed": [2020, 4, 14, 16, 7, 0, 1, 105, 0], "title": "7 Cloud Software Stocks to Buy \u2014 and 5 to Avoid \u2014 in the Covid-19 Era", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "7 Cloud Software Stocks to Buy \u2014 and 5 to Avoid \u2014 in the Covid-19 Era"}}, {"summary": "DOW UPDATE Shares of Home Depot and Johnson & Johnson are trading higher Tuesday morning, propelling the Dow Jones Industrial Average rally. The Dow (DJIA) was most recently trading 401 points, or 1.7%, higher, as shares of Home Depot (HD) and Johnson & Johnson (JNJ) have contributed to the index&apos;s intraday rally.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "DOW UPDATE Shares of Home Depot and Johnson & Johnson are trading higher Tuesday morning, propelling the Dow Jones Industrial Average rally. The Dow (DJIA) was most recently trading 401 points, or 1.7%, higher, as shares of Home Depot (HD) and Johnson & Johnson (JNJ) have contributed to the index&apos;s intraday rally."}, "id": "ba614b68-120e-3b68-b8e2-80139869da48", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/ba614b68-120e-3b68-b8e2-80139869da48/home-depot%2C-johnson-%26-johnson.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/ba614b68-120e-3b68-b8e2-80139869da48/home-depot%2C-johnson-%26-johnson.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 15:56:00 +0000", "published_parsed": [2020, 4, 14, 15, 56, 0, 1, 105, 0], "title": "Home Depot, Johnson & Johnson share gains contribute to Dow&#39;s 400-point rally", "title_detail": {"type": "text/html", "language": null, "base": "", "value": "Home Depot, Johnson & Johnson share gains contribute to Dow&#39;s 400-point rally"}}, {"summary": "(Bloomberg Opinion) -- The videoconferencing app Zoom has been one of the standout business successes of the Covid-19 pandemic, as millions of people around the world have been compelled to work and socialize from home.Research for Bloomberg Opinion by Apptopia indicates that, since early March, Zoom\u2019s downloads have dwarfed competition from well-established players such as Google, Microsoft and Skype.However, Zoom\u2019s meteoric rise arrived with a constellation of privacy and security concerns \u2014 not least \u201cZoombombing,\u201d in which hackers hijack video calls to harass and abuse users.It remains to be seen if Zoom can overcome these issues to secure a stable foothold in an increasingly lucrative market, or whether Covid-19 will prove the app to be a flash in the pan, discarded when the crisis has passed.\u00a0This column does not necessarily reflect the opinion of Bloomberg LP and its owners.Ben Schott is a Bloomberg Opinion visual columnist. He created the\u00a0Schott\u2019s Original Miscellany\u00a0and\u00a0Schott\u2019s Almanac\u00a0series, and writes for newspapers and magazines around the world. For more articles like this, please visit us at bloomberg.com/opinionSubscribe now to stay ahead with the most trusted business news source.\u00a92020 Bloomberg L.P.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "(Bloomberg Opinion) -- The videoconferencing app Zoom has been one of the standout business successes of the Covid-19 pandemic, as millions of people around the world have been compelled to work and socialize from home.Research for Bloomberg Opinion by Apptopia indicates that, since early March, Zoom\u2019s downloads have dwarfed competition from well-established players such as Google, Microsoft and Skype.However, Zoom\u2019s meteoric rise arrived with a constellation of privacy and security concerns \u2014 not least \u201cZoombombing,\u201d in which hackers hijack video calls to harass and abuse users.It remains to be seen if Zoom can overcome these issues to secure a stable foothold in an increasingly lucrative market, or whether Covid-19 will prove the app to be a flash in the pan, discarded when the crisis has passed.\u00a0This column does not necessarily reflect the opinion of Bloomberg LP and its owners.Ben Schott is a Bloomberg Opinion visual columnist. He created the\u00a0Schott\u2019s Original Miscellany\u00a0and\u00a0Schott\u2019s Almanac\u00a0series, and writes for newspapers and magazines around the world. For more articles like this, please visit us at bloomberg.com/opinionSubscribe now to stay ahead with the most trusted business news source.\u00a92020 Bloomberg L.P."}, "id": "34b67311-b79c-3c3c-b7ee-11252eaee661", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/news/giant-wfh-experiment-one-winner-155256661.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/news/giant-wfh-experiment-one-winner-155256661.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 15:52:56 +0000", "published_parsed": [2020, 4, 14, 15, 52, 56, 1, 105, 0], "title": "The Giant WFH Experiment Has\u00a0One Winner", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "The Giant WFH Experiment Has\u00a0One Winner"}}, {"summary": "The cloud software leader cleared an aggressive entry Tuesday, but it's still below a 175.10 double bottom buy point.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "The cloud software leader cleared an aggressive entry Tuesday, but it's still below a 175.10 double bottom buy point."}, "id": "5091599c-0cd2-3a6e-9f8b-23498c66aebd", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/video/microsoft-gaps-154042537.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/video/microsoft-gaps-154042537.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 15:40:42 +0000", "published_parsed": [2020, 4, 14, 15, 40, 42, 1, 105, 0], "title": "Microsoft Gaps Up", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "Microsoft Gaps Up"}}, {"summary": "Microsoft&#39;s (MSFT) investment in AI for Health is expected to ramp up development of COVID-19 cure and bodes well for its prospects in the healthcare space.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "Microsoft&#39;s (MSFT) investment in AI for Health is expected to ramp up development of COVID-19 cure and bodes well for its prospects in the healthcare space."}, "id": "355b792c-95e0-39e4-a071-f334c623127a", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/news/microsoft-msft-invests-ai-health-153803961.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/news/microsoft-msft-invests-ai-health-153803961.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 15:38:03 +0000", "published_parsed": [2020, 4, 14, 15, 38, 3, 1, 105, 0], "title": "Microsoft (MSFT) Invests in AI for Health to Fight COVID-19", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "Microsoft (MSFT) Invests in AI for Health to Fight COVID-19"}}, {"summary": "(Bloomberg) -- SAP SE, Europe\u2019s biggest tech company by market value, is exploring the sale of a unit that helps telecommunications operators route calls and text messages, people with knowledge of the matter said.The German firm is working with an adviser to gauge interest in the business and has been speaking to potential buyers, said the people, who asked not to be identified because the matter is private.SAP\u2019s digital interconnect business makes software that allows customers to communicate via phone or text on different carrier networks and processes related payments. The division traces its roots back to a company called Mobile 365 Inc., which Sybase Inc. acquired in 2006 for more than $400 million. It became part of SAP when the company bought Sybase in 2010.The business generates up to $50 million in annual earnings before interest, taxes, depreciation and amortization and about $250 million in revenue, the people said. It could fetch several hundred million dollars in a sale, they added.Some large technology companies including Microsoft Corp. and Intel Corp. have moved ahead with smaller deals despite the sharp downturn in activity due to the coronavirus pandemic.No final decisions have been made, and SAP could still decide to keep the business, the people said. A spokesman for SAP declined to comment.Divesting the legacy business would be in line with SAP\u2019s strategy to focus on growing its cloud products and integrating splashy acquisitions like Qualtrics International Inc., an $8 billion deal completed last year that was its largest ever.New co-chief executive officers, Christian Klein and Jennifer Morgan, must find ways to compete with younger companies like Salesforce.com Inc. and Workday Inc. given SAP\u2019s roots as a traditional enterprise software business. Bill McDermott, who stepped down as CEO in October, now leads ServiceNow Inc., a competitor in enterprise software.SAP has declined about 6% this year, valuing the Walldorf-based company at 139 billion euros ($152 billion). Shares were up Tuesday about 4%, outpacing Stoxx Europe 600 which rose 1.5%.(Updates fourth paragraph with details on valuation)For more articles like this, please visit us at bloomberg.comSubscribe now to stay ahead with the most trusted business news source.\u00a92020 Bloomberg L.P.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "(Bloomberg) -- SAP SE, Europe\u2019s biggest tech company by market value, is exploring the sale of a unit that helps telecommunications operators route calls and text messages, people with knowledge of the matter said.The German firm is working with an adviser to gauge interest in the business and has been speaking to potential buyers, said the people, who asked not to be identified because the matter is private.SAP\u2019s digital interconnect business makes software that allows customers to communicate via phone or text on different carrier networks and processes related payments. The division traces its roots back to a company called Mobile 365 Inc., which Sybase Inc. acquired in 2006 for more than $400 million. It became part of SAP when the company bought Sybase in 2010.The business generates up to $50 million in annual earnings before interest, taxes, depreciation and amortization and about $250 million in revenue, the people said. It could fetch several hundred million dollars in a sale, they added.Some large technology companies including Microsoft Corp. and Intel Corp. have moved ahead with smaller deals despite the sharp downturn in activity due to the coronavirus pandemic.No final decisions have been made, and SAP could still decide to keep the business, the people said. A spokesman for SAP declined to comment.Divesting the legacy business would be in line with SAP\u2019s strategy to focus on growing its cloud products and integrating splashy acquisitions like Qualtrics International Inc., an $8 billion deal completed last year that was its largest ever.New co-chief executive officers, Christian Klein and Jennifer Morgan, must find ways to compete with younger companies like Salesforce.com Inc. and Workday Inc. given SAP\u2019s roots as a traditional enterprise software business. Bill McDermott, who stepped down as CEO in October, now leads ServiceNow Inc., a competitor in enterprise software.SAP has declined about 6% this year, valuing the Walldorf-based company at 139 billion euros ($152 billion). Shares were up Tuesday about 4%, outpacing Stoxx Europe 600 which rose 1.5%.(Updates fourth paragraph with details on valuation)For more articles like this, please visit us at bloomberg.comSubscribe now to stay ahead with the most trusted business news source.\u00a92020 Bloomberg L.P."}, "id": "a9600f76-4a0d-3ad8-b1b6-9a975d1133e1", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/news/sap-said-explore-sale-mobile-122401506.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/news/sap-said-explore-sale-mobile-122401506.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 15:08:42 +0000", "published_parsed": [2020, 4, 14, 15, 8, 42, 1, 105, 0], "title": "SAP Is Said to Explore Sale of Mobile Network Software Unit", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "SAP Is Said to Explore Sale of Mobile Network Software Unit"}}, {"summary": "China&#39;s COVID-19 outbreak weighed on both PC production and demand in Q1. But sales got a boost late in the quarter.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "China&#39;s COVID-19 outbreak weighed on both PC production and demand in Q1. But sales got a boost late in the quarter."}, "id": "5caa2252-7aa5-3c49-b3a1-e8839951a4d9", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/5caa2252-7aa5-3c49-b3a1-e8839951a4d9/%5Bvideo%5Dpc-demand-isn%26%2339%3Bt.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/5caa2252-7aa5-3c49-b3a1-e8839951a4d9/%5Bvideo%5Dpc-demand-isn%26%2339%3Bt.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 10:00:00 +0000", "published_parsed": [2020, 4, 14, 10, 0, 0, 1, 105, 0], "title": "[video]PC Demand Isn&#39;t Quite as Bad as the Latest Shipment Estimates Would Suggest", "title_detail": {"type": "text/html", "language": null, "base": "", "value": "[video]PC Demand Isn&#39;t Quite as Bad as the Latest Shipment Estimates Would Suggest"}}, {"summary": "In response to a nationwide shortage of personal protective equipment, or PPE, for medical staff on the front lines of the fight against COVID-19, a coalition of private and nonprofit organizations have banded together to launch Protecting People Everywhere, an initiative powered by the HealthEquip\u2122 app through which PPE donors and their donations can be matched with potential recipients. The app ensures the equitable distribution of critical PPE and other medical supplies to the places they are needed most. The coalition includes the American Hospital Association, Kaiser Permanente, Kearney, Merit Solutions, Microsoft, and UPS. From technology and shipping to hospital advocacy and health care expertise, coalition members have worked together to solve an issue that is key to virus containment.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "In response to a nationwide shortage of personal protective equipment, or PPE, for medical staff on the front lines of the fight against COVID-19, a coalition of private and nonprofit organizations have banded together to launch Protecting People Everywhere, an initiative powered by the HealthEquip\u2122 app through which PPE donors and their donations can be matched with potential recipients. The app ensures the equitable distribution of critical PPE and other medical supplies to the places they are needed most. The coalition includes the American Hospital Association, Kaiser Permanente, Kearney, Merit Solutions, Microsoft, and UPS. From technology and shipping to hospital advocacy and health care expertise, coalition members have worked together to solve an issue that is key to virus containment."}, "id": "424ecd60-0a17-379e-a3f6-11587f2d5b65", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/news/coalition-organizations-launch-protecting-people-090000281.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/news/coalition-organizations-launch-protecting-people-090000281.html?.tsrc=rss", "published": "Tue, 14 Apr 2020 09:00:00 +0000", "published_parsed": [2020, 4, 14, 9, 0, 0, 1, 105, 0], "title": "Coalition of organizations launch the &#39;Protecting People Everywhere&#39; initiative answering the call to source safe and effective PPE for front-line health care workers", "title_detail": {"type": "text/html", "language": null, "base": "", "value": "Coalition of organizations launch the &#39;Protecting People Everywhere&#39; initiative answering the call to source safe and effective PPE for front-line health care workers"}}, {"summary": "In the midst of an economic scare as severe as this one, investors should look to blue-chip stocks to weather the storm. Many of these companies have massive financial resources that will allow them to not only keep the lights on while the economy grinds to a standstill, but get aggressive on the upswing, through advertising spend, research and even acquisitions.I recently wrote about the importance of balance sheets to investors. You should always consider factors such as a company&#39;s cash and debt before buying. But doing so is all the more vital when Wall Street&#39;s focus shifts from growth to survivability.You see an affinity for blue chips in the analyst community. For instance, Jefferies, in a March 16 report outlining high-quality stocks to buy on the dip, was focused on &quot;strong business models, healthy cash flow and very robust balance sheets.&quot; That&#39;s a common refrain among the pros finally emerging to suggest buying this dip.Here are 25 blue-chip stocks with some of the strongest balance sheets on Wall Street. Each of these stocks&#39; balance sheets are graded on a 1-20 scale (the higher, the better). We also included two other measures: a &quot;market-based rating&quot; and a &quot;COVID sensitivity score.&quot; The market-based rating takes bond-market signals into account. For instance, an unusually high yield on a bond indicates that fixed-income investors see a higher risk of default. (I&#39;ve always said bond investors are some of the smartest minds in the market.) Meanwhile, the COVID sensitivity score measures the impact of the coronavirus outbreak to a variety of income and balance-sheet items.Both scores are in a range from 1-5; again, the higher the better. So a market-based rating of 5 is a strong signal of confidence from the bond market. And a COVID sensitivity score of 5 implies the stock is much less sensitive to the outbreak than your average company.One last note before we dig in: A balance sheet is a snapshot of a moment in time. Therefore, if you&#39;re going to use balance sheets to educate your investments, you&#39;ll need to keep looking at the most recently available reports.Now, let&#39;s look at these well-fortified blue chips. SEE ALSO: 20 Best Stocks to Buy Now for the Next Bull Market", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "In the midst of an economic scare as severe as this one, investors should look to blue-chip stocks to weather the storm. Many of these companies have massive financial resources that will allow them to not only keep the lights on while the economy grinds to a standstill, but get aggressive on the upswing, through advertising spend, research and even acquisitions.I recently wrote about the importance of balance sheets to investors. You should always consider factors such as a company&#39;s cash and debt before buying. But doing so is all the more vital when Wall Street&#39;s focus shifts from growth to survivability.You see an affinity for blue chips in the analyst community. For instance, Jefferies, in a March 16 report outlining high-quality stocks to buy on the dip, was focused on &quot;strong business models, healthy cash flow and very robust balance sheets.&quot; That&#39;s a common refrain among the pros finally emerging to suggest buying this dip.Here are 25 blue-chip stocks with some of the strongest balance sheets on Wall Street. Each of these stocks&#39; balance sheets are graded on a 1-20 scale (the higher, the better). We also included two other measures: a &quot;market-based rating&quot; and a &quot;COVID sensitivity score.&quot; The market-based rating takes bond-market signals into account. For instance, an unusually high yield on a bond indicates that fixed-income investors see a higher risk of default. (I&#39;ve always said bond investors are some of the smartest minds in the market.) Meanwhile, the COVID sensitivity score measures the impact of the coronavirus outbreak to a variety of income and balance-sheet items.Both scores are in a range from 1-5; again, the higher the better. So a market-based rating of 5 is a strong signal of confidence from the bond market. And a COVID sensitivity score of 5 implies the stock is much less sensitive to the outbreak than your average company.One last note before we dig in: A balance sheet is a snapshot of a moment in time. Therefore, if you&#39;re going to use balance sheets to educate your investments, you&#39;ll need to keep looking at the most recently available reports.Now, let&#39;s look at these well-fortified blue chips. SEE ALSO: 20 Best Stocks to Buy Now for the Next Bull Market"}, "id": "fcd6c433-1c13-3b39-bc54-0f44900da9f4", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/news/25-blue-chips-brawny-balance-202243466.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/news/25-blue-chips-brawny-balance-202243466.html?.tsrc=rss", "published": "Mon, 13 Apr 2020 21:49:59 +0000", "published_parsed": [2020, 4, 13, 21, 49, 59, 0, 104, 0], "title": "25 Blue Chips With Brawny Balance Sheets", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "25 Blue Chips With Brawny Balance Sheets"}}, {"summary": "The stock closed just short of a record as investors responded to a flurry of signs of strengthening growth despite the economic shutdown triggered by the coronavirus pandemic.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "The stock closed just short of a record as investors responded to a flurry of signs of strengthening growth despite the economic shutdown triggered by the coronavirus pandemic."}, "id": "567a94b5-5881-3f35-b6d2-ddd80c12a66e", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/567a94b5-5881-3f35-b6d2-ddd80c12a66e/amazon-stock-takes-off-as.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/567a94b5-5881-3f35-b6d2-ddd80c12a66e/amazon-stock-takes-off-as.html?.tsrc=rss", "published": "Mon, 13 Apr 2020 20:15:00 +0000", "published_parsed": [2020, 4, 13, 20, 15, 0, 0, 104, 0], "title": "Amazon Stock Takes Off as Demand Surges in a Stay-at-Home World", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "Amazon Stock Takes Off as Demand Surges in a Stay-at-Home World"}}, {"summary": "The best tech stocks to buy and watch are strong price performers with healthy fundamentals, thanks to a new product or service that&#39;s driving growth.", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "The best tech stocks to buy and watch are strong price performers with healthy fundamentals, thanks to a new product or service that&#39;s driving growth."}, "id": "6fa55a71-2a5c-3c7c-852c-8da7ea2d36ba", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/6fa55a71-2a5c-3c7c-852c-8da7ea2d36ba/best-tech-stocks-to-buy-or.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/6fa55a71-2a5c-3c7c-852c-8da7ea2d36ba/best-tech-stocks-to-buy-or.html?.tsrc=rss", "published": "Mon, 13 Apr 2020 18:20:29 +0000", "published_parsed": [2020, 4, 13, 18, 20, 29, 0, 104, 0], "title": "Best Tech Stocks To Buy Or Watch Now: 5 Growth Stocks With Leadership Potential", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "Best Tech Stocks To Buy Or Watch Now: 5 Growth Stocks With Leadership Potential"}}, {"summary": "IBD Live Q&A Question Answer(s) Good Morning IBDTeam Livers! Good Morning Georgia GM Steve! Good morning all! Hey Blaine! good morning GM John Good morning. GM Scott Good morning all of you. GM Taher, thanks for joining us! Good morning. does any one follow the Zwieg thrust signal and if so where to go to get the status of it....", "summary_detail": {"type": "text/html", "language": null, "base": "", "value": "IBD Live Q&A Question Answer(s) Good Morning IBDTeam Livers! Good Morning Georgia GM Steve! Good morning all! Hey Blaine! good morning GM John Good morning. GM Scott Good morning all of you. GM Taher, thanks for joining us! Good morning. does any one follow the Zwieg thrust signal and if so where to go to get the status of it...."}, "id": "c970a0da-613e-3565-88ec-d4ab5ffddd97", "guidislink": false, "links": [{"rel": "alternate", "type": "text/html", "href": "https://finance.yahoo.com/m/c970a0da-613e-3565-88ec-d4ab5ffddd97/ibd-live-q%26a-and-two-key.html?.tsrc=rss"}], "link": "https://finance.yahoo.com/m/c970a0da-613e-3565-88ec-d4ab5ffddd97/ibd-live-q%26a-and-two-key.html?.tsrc=rss", "published": "Mon, 13 Apr 2020 16:22:24 +0000", "published_parsed": [2020, 4, 13, 16, 22, 24, 0, 104, 0], "title": "IBD Live Q&A And Two Key Stock Lists For April 13, 2020", "title_detail": {"type": "text/plain", "language": null, "base": "", "value": "IBD Live Q&A And Two Key Stock Lists For April 13, 2020"}}]
'''

# pprint.pprint(f['entries'][0])
# j = json.dumps(f['entries'][0])
entries = json.loads(a)
len(entries)
entries[0]


{'summary': 'Futures fell early Wednesday as Tesla and airlines rallied. The coronavirus stock rally soared Tuesday as Amazon, Microsoft and Nvidia led breakouts. Apple and the Nasdaq cleared key levels.',
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': '',
  'value': 'Futures fell early Wednesday as Tesla and airlines rallied. The coronavirus stock rally soared Tuesday as Amazon, Microsoft and Nvidia led breakouts. Apple and the Nasdaq cleared key levels.'},
 'id': '8288ce8e-17d6-3d23-9871-7adf026869b5',
 'guidislink': False,
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://finance.yahoo.com/m/8288ce8e-17d6-3d23-9871-7adf026869b5/dow-jones-futures-fall-but.html?.tsrc=rss'}],
 'link': 'https://finance.yahoo.com/m/8288ce8e-17d6-3d23-9871-7adf026869b5/dow-jones-futures-fall-but.html?.tsrc=rss',
 'published': 'Wed, 15 Apr 2020 11:39:02 +0000',
 'published_parsed': [2020, 4, 15, 11, 39, 2, 2, 106, 0],
 'title': 'Dow Jones Futures Fall But T

In [22]:
import datetime
import json
t = [2020, 4, 14, 21, 35, 43, 1, 105, 0]

# json.dumps(datetime.datetime(*t[:7]))

TypeError: Object of type datetime is not JSON serializable

In [23]:
from time import mktime
import datetime
 
fetched_at = datetime.datetime(2020, 4, 12, 10, 0, 0)
freq = datetime.timedelta(seconds=12000)
print(fetched_at + freq)
secs_to_sleep = (fetched_at + freq - datetime.datetime.now()).total_seconds()

if secs_to_sleep > 0:
    print("sleep for {} seconds".format(int(secs_to_sleep)))
else:
    print("scrape")

2020-04-12 13:20:00
sleep for 5955 seconds


In [8]:
from pprint import pprint
import aiohttp
import datetime
import feedparser
from newspaper import Article, ArticleException

async def fetch(item):
    try:
        async with aiohttp.ClientSession() as sess:
            async with sess.get(item['link']) as resp:
                html = await resp.text()
    except aiohttp.ClientError as e:
        log.error(e)
        html = None

    data = dict(
        url=item['link'],
        rss_title=item['title'],
        rss_summary=item['summary'],
        rss_published_at=item['published_parsed'],
#         rss_tickers=entry['ticker'],
#         raw=html,
        fetched_at=datetime.datetime.now())
    try:
        article = Article(item['link'])
        article.set_html(html)
        article.parse()
        data = dict(**data,
                    parsed_title=article.title,
                    parsed_text=article.text,
                    parsed_published_at=article.publish_date)
    except ArticleException as e:
        log.debug(e)
    finally:
#         page = Page.create(**data)
        pprint(data)


# url = 'https://finance.yashoo.com/m/92be1bf4-f6e5-3659-95bf-612044770400/disney-world-is-furloughing.html?.tsrc=rss'
f = feedparser.parse('http://finance.yahoo.com/rss/headline?s=dis')
await fetch(f['entries'][0])

# try:
#     article = Article("url")
#     article.set_html(None)
#     article.parse()
#     article.nlp()
#     print(article.authors)
#     print(article.publish_date)
#     print(article.text)
#     print(article.keywords)
#     print(article.summary)
# except ArticleException as e:
#     print(e)

{'fetched_at': datetime.datetime(2020, 4, 15, 9, 35, 12, 79073),
 'parsed_published_at': datetime.datetime(2020, 4, 14, 21, 35, 43, tzinfo=tzlocal()),
 'parsed_text': 'Is Disney Stock A Buy Right Now, As Iger Resumes Control Amid '
                'Coronavirus Crisis? More\n'
                '\n'
                'Disney stock is well off its highs as coronavirus takes a '
                "toll. Here's what fundamental and technical analysis says "
                'about buying Disney now.',
 'parsed_title': 'Is Disney Stock A Buy Right Now, As Iger Resumes Control '
                 'Amid Coronavirus Crisis?',
 'raw': '<!DOCTYPE html><html id="atomic" class="NoJs featurephone" '
        'lang="en-US"><head prefix="og: '
        'http://ogp.me/ns#"><script>window.performance && '
        'window.performance.mark && '
        "window.performance.mark('PageStart');</script><meta "
        'charset="utf-8"/><meta name="oath:guce:consent-host" '
        'content="guce.yahoo.com"/><meta name

        'style="padding-bottom:52%;background-image:url(https://s.yimg.com/ny/api/res/1.2/GYRjqLnSAhr9y87TASrNkg--~A/YXBwaWQ9aGlnaGxhbmRlcjtoPTMxMjt3PTYwMDtxPTc1O2ZpPXN0cmlt/http://l1.yimg.com/uu/api/res/1.2/eiMndcYv_b5XeXGPqeM4Yw--/YXBwaWQ9eXRhY2h5b247cT03NTs-/https://media-mbst-pub-ue1.s3.amazonaws.com/creatr-images/2020-04/697f9000-7dbd-11ea-9fbc-cb14f1bd4226);" '
        'data-reactid="72"></div></div><h3 class="M(0) Py(1px)" '
        'data-reactid="73"><a class="Fz(13px) LineClamp(4,96px) C(#0078ff):h '
        'Td(n) C($c-fuji-blue-4-b) smartphone_C(#000) smartphone_Fz(19px)" '
        'href="https://finance.yahoo.com/news/amazon-lawsuit-over-10-billion-jedi-contract-145924302.html" '
        'data-reactid="74"><span class="Fw(600) smartphone_Fw(500)" '
        'data-reactid="75">Amazon’s lawsuit over a $10 billion Pentagon '
        'contract lays out disturbing allegations against Trump</span><u '
        'class="StretchedBox Z(1)" data-reactid="76"></u></a></h3><div '
       

        'action.","customTitleTag":"Women\'s empowerment content and '
        'conference - '
        'MAKERS"},"sectionLifeTag":{"path":"\\u002F:contentSite(lifestyle)\\u002Ftagged\\u002F:tag","action":"loadConfigAndPage","method":"get","page":"magtag","pageType":"magtag","renderTarget":"default","useYqlCTopic":true,"enableContentRedirect":true,"spaceid":980751755},"lifestylepreview":{"path":"\\u002F:contentSite(lifestyle)(\\u002Fm)?\\u002F:uuid([a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12})\\u002F:alias","action":"loadConfigAndPage","contentSite":"lifestyle","contentType":"story","method":"get","page":"content","pageType":"content"},"LifeSlideshow":{"path":"\\u002F:contentSite(lifestyle)\\u002F:type(photos|ss)\\u002F:alias","method":"get","page":"content","action":"loadConfigAndPage","pageType":"content"},"LifeSlideshowPhoto":{"path":"\\u002F:contentSite(lifestyle)\\u002Fphotos\\u002F:alias\\u002F:photoAlias.html","method":"get","page":"content","action

        '147.25 flat-base buy point in torrid volume, hitting a record high. '
        'It then eased to slip below the proper '
        'entry.","length":185,"tagName":"p"},{"type":"text","content":"The '
        'stock briefly retook the 147.25 buy point before easing again. It had '
        'been working on a new flat base with a 153.51 buy point, \\u003Ca '
        'href=\\"https:\\u002F\\u002Fmarketsmith.investors.com\\u002F\\" '
        'rel=\\"nofollow noopener\\" target=\\"_blank\\"\\u003EMarketSmith '
        'chart analysis\\u003C\\u002Fa\\u003E shows. Disney stock gapped down '
        'below its 200-day line on Feb. 24 in the broader market sell-off '
        'fueled by coronavirus cases spreading outside of '
        'China.","length":310,"tagName":"p"},{"type":"text","content":"It fell '
        'nearly 4% Feb. 25. Following the Iger news, shares lost another 4% '
        'the next session. With the correction from the left-side high to the '
        'bottom at 24%, Disne

In [15]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://es:9200")
es.info()

{'name': '66dfbd372aa5',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'i_0M-3txTu6W9VNq8rpZXw',
 'version': {'number': '7.6.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'aa751e09be0a5072e8570670309b1f12348f023b',
  'build_date': '2020-02-29T00:15:25.529771Z',
  'build_snapshot': False,
  'lucene_version': '8.4.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [28]:
import twint

c = twint.Config()

# c.Username = "YahooFinance"
# c.Username = "CNBC"
c.Search = '$AAPL'
c.Limit = 10
# c.Since= '2017-12-27'
# c.Until = '2017-12-31'
# c.Store_csv = True
# c.Output = "none"
c.Elasticsearch = "http://es:9200"
c.Debug = True

twint.run.Search(c)

[+] Indexing to Elasticsearch @ http://es:9200
....................

In [12]:
c